In [39]:
from azureml.core import Workspace, Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace, Webservice

ws = Workspace.from_config()
service_name = "catboost-endpoint"

try:
    service = Webservice(ws, service_name)
    service.delete()
    print(f"🗑️ Deleted existing service: {service_name}")
except:
    print(f"⚠️ No existing service found: {service_name}")

# ✅ Confirm model exists
model_name = "catboost-thrombosis-predictor"
model = Model(ws, model_name)
print(f"✅ Found registered model: {model_name}")


⚠️ No existing service found: catboost-endpoint
✅ Found registered model: catboost-thrombosis-predictor


In [35]:
env = Environment(name="catboost-env")
env.python.conda_dependencies.add_pip_package("catboost")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("azureml-defaults")
print("✅ Environment created successfully")


✅ Environment created successfully


In [36]:
inference_config = InferenceConfig(
    entry_script="score.py",  # Ensure this file exists
    environment=env
)
print("✅ Inference configuration created")


✅ Inference configuration created


In [37]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=False
)
print("✅ Deployment configuration set")


✅ Deployment configuration set


In [38]:
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


/tmp/ipykernel_3110/4183800143.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


WebserviceException: WebserviceException:
	Message: Service catboost-endpoint with the same name already exists, please use a different service name or delete the existing service.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service catboost-endpoint with the same name already exists, please use a different service name or delete the existing service."
    }
}

In [44]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("✅ Azure ML Workspace Loaded:", ws.name)


✅ Azure ML Workspace Loaded: mlops-simple-flow


In [45]:
from azureml.core.model import Model

model = Model.register(
    workspace=ws,
    model_name="catboost-thrombosis-predictor",
    model_path="catboost_model.cbm",  # Path to your model file
    description="CatBoost model for predicting thrombotic events",
)

print("✅ Model registered successfully!")


Registering model catboost-thrombosis-predictor
✅ Model registered successfully!


In [46]:
env_yaml = """name: catboost-env
dependencies:
  - python=3.8
  - pip:
    - azureml-defaults
    - catboost
    - numpy
    - pandas
"""
with open("environment.yml", "w") as f:
    f.write(env_yaml)

print("✅ environment.yml created successfully!")


✅ environment.yml created successfully!


In [47]:
from azureml.core import Environment

env = Environment.from_conda_specification(name="catboost-env", file_path="environment.yml")
env.register(workspace=ws)

print("✅ Environment registered successfully!")


✅ Environment registered successfully!


In [48]:
score_script = """import json
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
from catboost import CatBoostClassifier

def init():
    global model
    model_path = Model.get_model_path("catboost-thrombosis-predictor")
    model = CatBoostClassifier()
    model.load_model(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        df = pd.DataFrame(data)
        
        # Convert categorical features to int if necessary
        categorical_features = ["gender", "work_type", "Residence_type", "smoking_status"]
        for col in categorical_features:
            if col in df.columns:
                df[col] = df[col].astype(str).astype("category")

        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}
"""

with open("score.py", "w") as f:
    f.write(score_script)

print("✅ score.py created successfully!")


✅ score.py created successfully!
